In [7]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [8]:
import numpy as np
import pandas as pd

In [9]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [52]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [53]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [54]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [55]:
# Collect the names of tables within the database
inspector.get_table_names()

['measurement', 'station']

In [56]:
# Using the inspector to print the column names within the 'measurement' table and its types
columns = inspector.get_columns('measurement')
for c in columns:
    print(c["name"], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [57]:
# Using the inspector to print the column names within the 'station' table and its types
columns = inspector.get_columns('station')
for c in columns:
    print(c["name"], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [31]:
# Declare a Base using 'automap_base()'
Base = automap_base()

In [32]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [33]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['measurement', 'station']

In [34]:
# Assign the measurement and station classes to variables called 'Measurement' and 'Station'
Measurement = Base.classes.measurement
Station = Base.classes.station

In [35]:
# Create a session
session = Session(engine)

In [36]:
# Display the first row's columns and data in dictionary format for Measurement
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd2c8140dd8>,
 'prcp': 0.08,
 'station': 'USC00519397',
 'id': 1,
 'tobs': 65.0,
 'date': '2010-01-01'}

In [37]:
# Display the first row's columns and data in dictionary format for Station
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd2c815b400>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'id': 1,
 'elevation': 3.0,
 'latitude': 21.2716,
 'station': 'USC00519397'}

In [ ]:




# reflect an existing database into a new model

# reflect the tables

In [ ]:
# We can view all of the classes that automap found


In [ ]:
# Save references to each table


In [ ]:
# Create our session (link) from Python to the DB


# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [58]:
# Query database to find the last date of precipitation data available(last datapoint) 
engine.execute('SELECT MAX(date) FROM measurement').fetchall()

[('2017-08-23',)]

In [68]:
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [69]:
query_date = dt.date(2017, 8,23) - dt.timedelta(days=365)
print(query_date)

2016-08-23


In [94]:
date = dt.datetime(2016, 8, 22)

results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > date).order_by(Measurement.date).all()
#results                                                                  
data = list(np.ravel(results))
data                                                                 

['2016-08-23',
 0.0,
 '2016-08-23',
 0.15,
 '2016-08-23',
 0.05,
 '2016-08-23',
 None,
 '2016-08-23',
 0.02,
 '2016-08-23',
 1.79,
 '2016-08-23',
 0.7,
 '2016-08-24',
 0.08,
 '2016-08-24',
 2.15,
 '2016-08-24',
 2.28,
 '2016-08-24',
 None,
 '2016-08-24',
 1.22,
 '2016-08-24',
 2.15,
 '2016-08-24',
 1.45,
 '2016-08-25',
 0.08,
 '2016-08-25',
 0.08,
 '2016-08-25',
 0.0,
 '2016-08-25',
 0.0,
 '2016-08-25',
 0.21,
 '2016-08-25',
 0.06,
 '2016-08-25',
 0.11,
 '2016-08-26',
 0.0,
 '2016-08-26',
 0.03,
 '2016-08-26',
 0.02,
 '2016-08-26',
 0.04,
 '2016-08-26',
 0.0,
 '2016-08-26',
 0.01,
 '2016-08-27',
 0.0,
 '2016-08-27',
 0.18,
 '2016-08-27',
 0.02,
 '2016-08-27',
 0.0,
 '2016-08-27',
 0.12,
 '2016-08-27',
 None,
 '2016-08-28',
 0.01,
 '2016-08-28',
 0.14,
 '2016-08-28',
 0.14,
 '2016-08-28',
 0.14,
 '2016-08-28',
 0.6,
 '2016-08-28',
 2.07,
 '2016-08-29',
 0.0,
 '2016-08-29',
 0.17,
 '2016-08-29',
 0.04,
 '2016-08-29',
 None,
 '2016-08-29',
 0.0,
 '2016-08-29',
 0.35,
 '2016-08-29',
 0.9,


In [127]:
df = pd.DataFrame(results, columns=['date', 'prcp'])
df.set_index('date', inplace=True, )
#plt.figure(figsize=(10,20))
df.head(15)
#df.plot.bar()
#plt.tight_layout()
#plt.show()

,prcp
date,
2016-08-23,0.00
2016-08-23,0.15
2016-08-23,0.05
2016-08-23,NaN
2016-08-23,0.02
2016-08-23,1.79
2016-08-23,0.70
2016-08-24,0.08
2016-08-24,2.15


In [76]:
# Use the session to query 

In [128]:
# Use Pandas to calculate the summary statistics for the precipitation data
group_summary_df = df.groupby("date").agg(["mean", "median","var","std", "sem"])
group_summary_df.head()


prcp                                     
                mean median       var       std       sem
date                                                     
2016-08-23  0.451667  0.100  0.499097  0.706468  0.288414
2016-08-24  1.555000  1.800  0.706510  0.840541  0.343150
2016-08-25  0.077143  0.080  0.005157  0.071813  0.027143
2016-08-26  0.016667  0.015  0.000267  0.016330  0.006667
2016-08-27  0.064000  0.020  0.006680  0.081731  0.036551

In [119]:
# Design a query to list all of the stations
list_station = session.query(Station.station).all()
list_station

[('USC00519397'),
 ('USC00513117'),
 ('USC00514830'),
 ('USC00517948'),
 ('USC00518838'),
 ('USC00519523'),
 ('USC00519281'),
 ('USC00511918'),
 ('USC00516128')]

In [120]:
# Design a query to show how many stations are available in this dataset?
number = session.query(func.count(Station.station)).all()
#number

print(f'Number of Stations:{number[0]}')

Number of Stations:(9,)


In [122]:
# Design a query to list all of the station names
list_name = session.query(Station.name).all()
list_name

[('WAIKIKI 717.2, HI US'),
 ('KANEOHE 838.1, HI US'),
 ('KUALOA RANCH HEADQUARTERS 886.9, HI US'),
 ('PEARL CITY, HI US'),
 ('UPPER WAHIAWA 874.3, HI US'),
 ('WAIMANALO EXPERIMENTAL FARM, HI US'),
 ('WAIHEE 837.5, HI US'),
 ('HONOLULU OBSERVATORY 702.2, HI US'),
 ('MANOA LYON ARBO 785.2, HI US')]

In [121]:
# Query to count all the station names
name_station = session.query(func.count(Station.name)).all()
name_station

[(9)]

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
